# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Horn Satisfiability

In [1]:
import time

In [2]:
from collections import defaultdict, deque

# Algorithm

In [3]:
%%time
def knowledge_base(formulas):
    rules, variable, dependency = [], defaultdict(bool), defaultdict(list)

    def _clause(formula):
        # A, B, C => P
        neg, pos = formula.replace(' ', '').split('=>')
        neg, pos = set(neg.split('&')) - {''}, pos or None

        # add rule
        rules.append((neg, pos))

        # set variable and track dependencies
        for i in neg:
            dependency[i].append((neg, pos))

    # parse formulas and build knowledge base
    deque((_clause(i) for i in formulas.split('\n') if i), 0)

    return rules, variable, dependency

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.15 µs


In [4]:
%%time
def resolution(rules, variable, dependency):
    # initial variables that have to be satisfied
    to_satisfy = [(neg, pos) for neg, pos in rules if not neg]

    while to_satisfy:
        neg, pos = to_satisfy.pop()

        # contradiction: true => false
        if not pos:
            return False

        # satisfy variable
        variable[pos] = True

        # update dependent rules
        for d_neg, d_pos in dependency[pos]:
            d_neg.remove(pos)

            # next variable to be satisfied
            if not d_neg and d_pos not in variable:
                to_satisfy.append((d_neg, d_pos))

    return True

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 12.2 µs


In [5]:
%%time
def hornsat(formulas):
    rules, variable, dependency = knowledge_base(formulas)
    satisfiable = resolution(rules, variable, dependency)

    print(['CONTRADICTION', 'SATISFIABLE'][satisfiable])
    print(', '.join('%s=%s' % i for i in variable.items()))

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11 µs


# Run

In [6]:
%%time
hornsat("""
X => Y
Y => Z
=> X
""")

SATISFIABLE
X=True, Y=True, Z=True
CPU times: user 1.9 ms, sys: 0 ns, total: 1.9 ms
Wall time: 1.92 ms


In [7]:
%%time
hornsat("""
X => Y
Y => X
=> X
Y =>
""")

CONTRADICTION
X=True, Y=True
CPU times: user 1.38 ms, sys: 854 µs, total: 2.23 ms
Wall time: 2.33 ms


In [8]:
%%time
hornsat("""
P & Q & R & S => X
P & Q => R
R => S
X =>
=> P
=> R
""")

SATISFIABLE
R=True, S=True, P=True
CPU times: user 123 µs, sys: 16 µs, total: 139 µs
Wall time: 145 µs


In [9]:
%%time
hornsat("""
P & Q & R & S => X
P & Q => R
R => S
X =>
=> P
=> Q
""")

CONTRADICTION
Q=True, P=True, R=True, S=True, X=True
CPU times: user 2.33 ms, sys: 0 ns, total: 2.33 ms
Wall time: 2.41 ms


# The End